In [1]:
%matplotlib nbagg
import matplotlib.pyplot as plt
import time
import numpy as np

import qcodes as qc

qc.set_mp_method('spawn')  # force Windows behavior on mac

# this makes a widget in the corner of the window to show and control
# subprocesses and any output they would print to the terminal
qc.show_subprocess_widget()

<IPython.core.display.Javascript object>

In [2]:
# spawn doesn't like function or class definitions in the interpreter
# session - had to move them to a file.
from toymodel import AModel, MockGates, MockSource, MockMeter, AverageGetter, AverageAndRaw

# now create this "experiment"
model = AModel()
gates = MockGates('gates', model=model)
source = MockSource('source', model=model)
meter = MockMeter('meter', model=model)

station = qc.Station(gates, source, meter)

# could measure any number of things by adding arguments to this
# function call, but here we're just measuring one, the meter amplitude
station.set_measurement(meter.amplitude)

# it's nice to have the key parameters be part of the global namespace
# that way they're objects that we can easily set, get, and slice
# this could be simplified to a station method that gathers all parameters
# and adds them all as (disambiguated) globals, printing what it did
# something like:
#   station.gather_parameters(globals())
c0, c1, c2, vsd = gates.chan0, gates.chan1, gates.chan2, source.amplitude

# once we have implemented a monitor, defining a station will start a
# DataServer process, and you would see it in the subprocess widget,
# or via active_children() as here:
# qc.active_children()

In [3]:
a = c0[2:0:0.5]
# a.reverse()
# a._values.reverse()
b = a + c0.sweep(9,7,1)
# b = a + a.__reversed__()
# a.__reversed__()._values
# b._values
for s in b.snapshot():
    print(s)
print(b._values)
b.reverse()
print()
for s in b.snapshot():
    print(s)
print(b._values)
b.reverse()
print()
for s in b.snapshot():
    print(s)
print(b._values)

{'type': 'slice', 'reverse': False, 'start': 2, 'step': 0.5, 'stop': 0}
{'type': 'sweep', 'reverse': False, 'start': 9, 'stop': 7, 'num': None, 'step': 1}
[2.0, 1.5, 1.0, 0.5, 9.0, 8.0, 7.0]

{'type': 'sweep', 'reverse': True, 'start': 9, 'stop': 7, 'num': None, 'step': 1}
{'type': 'slice', 'reverse': True, 'start': 2, 'step': 0.5, 'stop': 0}
[7.0, 8.0, 9.0, 0.5, 1.0, 1.5, 2.0]

{'type': 'slice', 'reverse': False, 'start': 2, 'step': 0.5, 'stop': 0}
{'type': 'sweep', 'reverse': False, 'start': 9, 'stop': 7, 'num': None, 'step': 1}
[2.0, 1.5, 1.0, 0.5, 9.0, 8.0, 7.0]


In [4]:
# start a Loop (which by default runs in a seprarate process)
# the sweep values are defined by slicing the parameter object
# but more complicated sweeps (eg nonlinear, or adaptive) can
# easily be used instead

# Notice that I'm using an explicit location and `overwrite=True` here so that
# running this notebook over and over won't result in extra files.
# But if you leave these out, you get a new timestamped DataSet each time.
data = qc.Loop(c0.sweep(0,10,0.2), 0.03).run(location='data/testsweep', overwrite=True)

# now there should be two extra processes running, DataServer and a sweep
# I'll omit the active_children call now because you can see them in the
# subprocess widget

DataSet: DataMode.PULL_FROM_SERVER, location='data/testsweep'
   amplitude: amplitude
   chan0: chan0
started at 2016-04-24 16:23:12


In [5]:
# manually bring the data into the main process and display it as numbers
data.sync()
data.arrays

{'amplitude': DataArray[51]: amplitude
 array([ 0.117,  0.115,  0.106,  0.092,  0.077,  0.064,  0.053,  0.044,
         0.037,  0.031,  0.027,  0.023,  0.02 ,  0.018,  0.016,  0.014,
         0.013,  0.011,  0.01 ,  0.01 ,  0.009,  0.008,  0.007,  0.007,
         0.006,  0.006,  0.006,  0.007,  0.007,  0.008,  0.009,  0.01 ,
         0.01 ,  0.011,  0.013,  0.014,  0.016,  0.018,  0.02 ,  0.023,
         0.027,  0.031,  0.037,  0.044,  0.053,  0.064,  0.077,  0.092,
         0.106,  0.115,  0.117]), 'chan0': DataArray[51]: chan0
 array([  0. ,   0.2,   0.4,   0.6,   0.8,   1. ,   1.2,   1.4,   1.6,
          1.8,   2. ,   2.2,   2.4,   2.6,   2.8,   3. ,   3.2,   3.4,
          3.6,   3.8,   4. ,   4.2,   4.4,   4.6,   4.8,   5. ,   5.2,
          5.4,   5.6,   5.8,   6. ,   6.2,   6.4,   6.6,   6.8,   7. ,
          7.2,   7.4,   7.6,   7.8,   8. ,   8.2,   8.4,   8.6,   8.8,
          9. ,   9.2,   9.4,   9.6,   9.8,  10. ])}

In [6]:
# live-updating plot, that syncs the data and stops updating when it's finished
# plot = qc.MatPlot(data.amplitude)
plotQ = qc.QtPlot(data.amplitude)

In [7]:
data2 = qc.Loop(c1.sweep(-15,15,1), 0.1).loop(c0.sweep(-15,15,0.5), 0.01).each(
    meter.amplitude, # first measurement, at c2=0 -> amplitude_0 bcs it's action 0
    qc.Task(c2.set, 1), # action 1 -> c2.set(1)
    qc.Wait(0.001),
    meter.amplitude, # second measurement, at c2=1 -> amplitude_4 bcs it's action 4
    qc.Task(c2.set, 0)
    ).run(location='test2d', overwrite=True)

# use the subplot and add features of qc.MatPlot
# plot2 = qc.MatPlot(data2.amplitude_0, cmap=plt.cm.hot, figsize=(12, 4.5), subplots=(1, 2))
# plot2.add(data2.amplitude_3, cmap=plt.cm.hot, subplot=2)

# the equivalent in QtPlot
plot2Q = qc.QtPlot(data2.amplitude_0, figsize=(1200, 500))
plot2Q.add(data2.amplitude_3, subplot=2)

DataSet: DataMode.PULL_FROM_SERVER, location='test2d'
   amplitude_3: amplitude
   chan1: chan1
   amplitude_0: amplitude
   chan0: chan0
started at 2016-04-24 16:23:21


In [9]:
data3 = qc.Loop(c1.sweep(-15,15,1), 0.1).each(
    qc.Task(c0.set, -10),
    qc.Task(c2.set, 0),
    # a 1D measurement
    meter.amplitude,
    # a 2D sweep, .each is actually unnecessary bcs this is the default measurement
    qc.Loop(c0.sweep(-15,15,1), 0.001).each(meter.amplitude),
    qc.Task(c0.set, -10),
    # a 2D sweep with the same outer but different inner loop
    qc.Loop(c2.sweep(-10,10,0.2), 0.001),
    AverageGetter(meter.amplitude, c2[-10:10:0.2], 0.001)
).run(location='data/test_multi_d', overwrite=True)

# several plots updating simultaneously
# plot3 = qc.MatPlot(data3.amplitude_3_0, cmap=plt.cm.hot)
# plot3b = qc.MatPlot(data3.amplitude_5_0, cmap=plt.cm.hot, figsize=(12, 4.5), subplots=(1,2))
# plot3b.add(data3.avg_amplitude, subplot=2)
plot3Q = qc.QtPlot(data3.amplitude_3_0)
plot3bQ = qc.QtPlot(data3.amplitude_5_0, figsize=(1200, 500))
plot3bQ.add(data3.avg_amplitude, subplot=2)

DataSet: DataMode.PULL_FROM_SERVER, location='data/test_multi_d'
   chan1: chan1
   amplitude_3_0: amplitude
   chan0: chan0
   avg_amplitude: avg_amplitude
   chan2: chan2
   amplitude_5_0: amplitude
   amplitude_2: amplitude
started at 2016-04-24 16:23:58


In [10]:
# An example of a parameter that returns several values of different dimension
# This produces the last two arrays from data3, but only takes the data once.
data4 = qc.Loop(c1.sweep(-15,15,1), 0.1).each(
    AverageAndRaw(meter.amplitude, c2.sweep(-10,10,0.2), 0.001)
).run(location='data/test_complex_param', overwrite=True)

# plot4 = qc.MatPlot(data4.amplitude, cmap=plt.cm.hot, subplots=(1,2), figsize=(12, 4.5))
# plot4.add(data4.avg_amplitude, subplot=2)
plot4Q = qc.QtPlot(data4.amplitude, figsize=(1200, 500))
plot4Q.add(data4.avg_amplitude, subplot=2)

DataSet: DataMode.PULL_FROM_SERVER, location='data/test_complex_param'
   amplitude: amplitude
   chan2: chan2
   chan1: chan1
   avg_amplitude: avg_amplitude
started at 2016-04-24 16:24:22
